In [1]:
# Import SparkSession
from pyspark.sql import SparkSession
from pyspark.sql.functions import datediff

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1,application_1588558552313_0002,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
# Create Spark session object
spark = SparkSession.builder.appName('data_processing').getOrCreate()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
# Load csv dataset
df = spark.read.csv('s3://tet-bd/datasets/spark/Casos_positivos_de_COVID-19_en_Colombia.csv', inferSchema=True, header=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
# Borrar columna 'Codigo DIVIPOLA' y 'FIS'
df = df.drop('Codigo DIVIPOLA', 'FIS')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
# Crear columna donde se almacena el calculo de dias transcurridos entre 'Fecha diagnostico'
# y 'fecha reporte web'
df = df.withColumn('dias_hasta_reporte', datediff(df['fecha reporte web'], df['Fecha diagnostico']))
df.filter(df['dias_hasta_reporte'] > 0).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+---------------------+-------------------+------------------------+------------+----+----+----------+--------+-------------------+---------------+-------------------+----------------+-------------------+------------------+
|ID de caso|Fecha de notificación|Ciudad de ubicación|Departamento o Distrito |    atención|Edad|Sexo|      Tipo|  Estado|País de procedencia|Fecha de muerte|  Fecha diagnostico|Fecha recuperado|  fecha reporte web|dias_hasta_reporte|
+----------+---------------------+-------------------+------------------------+------------+----+----+----------+--------+-------------------+---------------+-------------------+----------------+-------------------+------------------+
|      6010|  2020-04-26 00:00:00|Cartagena de Indias|     Cartagena D.T. y C.|Hospital UCI|  83|   F|En estudio|   Grave|           Colombia|          -   -|2020-04-28 00:00:00|            null|2020-04-29 00:00:00|                 1|
|      6011|  2020-04-26 00:00:00|Cartagena de Indias|     C

In [8]:
# Filtrar aquellos con 'Edad' < 50 y 'Sexo' = M
df.filter(df['Edad'] < 50).filter(df['Sexo'] == 'M').show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+---------------------+-------------------+------------------------+----------+----+----+-----------+------+--------------------+---------------+-------------------+--------------------+-------------------+------------------+
|ID de caso|Fecha de notificación|Ciudad de ubicación|Departamento o Distrito |  atención|Edad|Sexo|       Tipo|Estado| País de procedencia|Fecha de muerte|  Fecha diagnostico|    Fecha recuperado|  fecha reporte web|dias_hasta_reporte|
+----------+---------------------+-------------------+------------------------+----------+----+----+-----------+------+--------------------+---------------+-------------------+--------------------+-------------------+------------------+
|         2|  2020-03-06 00:00:00|Guadalajara de Buga|         Valle del Cauca|Recuperado|  34|   M|  Importado|  Leve|              España|          -   -|2020-03-09 00:00:00|2020-03-19T00:00:...|2020-03-09 00:00:00|                 0|
|         5|  2020-03-09 00:00:00|           Medellí

In [9]:
# Agrupar por 'Estado' y contar
df.groupBy('Estado').count().orderBy('count', ascending=False).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+-----+
|   Estado|count|
+---------+-----+
|     Leve| 5430|
| Moderado|  382|
|Fallecido|  278|
|    Grave|  117|
|     leve|    3|
|      N/A|    1|
+---------+-----+

In [10]:
# Guardar los resultados en el bucket de S3
write_uri = 's3://tet-bd/datasets/spark/resultados_covid'
df.coalesce(1).write.format('csv').option('header', 'true').save(write_uri)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
# Parquet
parquet_uri = 's3://tet-bd/datasets/spark/resultados_covid/parquet'
df.write.format('parquet').save(parquet_uri)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'Attribute name "ID de caso" contains invalid character(s) among " ,;{}()\\n\\t=". Please use alias to rename it.;'
Traceback (most recent call last):
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/sql/readwriter.py", line 738, in save
    self._jwrite.save(path)
  File "/usr/lib/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1257, in __call__
    answer, self.gateway_client, self.target_id, self.name)
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/sql/utils.py", line 69, in deco
    raise AnalysisException(s.split(': ', 1)[1], stackTrace)
pyspark.sql.utils.AnalysisException: 'Attribute name "ID de caso" contains invalid character(s) among " ,;{}()\\n\\t=". Please use alias to rename it.;'

